<a href="https://colab.research.google.com/github/Abyd0s/archi-helper/blob/main/BevGepTan_HorvathBalazs_SpotifyRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [270]:
import pandas as pd
from sklearn.compose import *
from sklearn.impute import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import *
from sklearn.ensemble import *
import pickle

In [271]:
# Mindenek előtt fontos feltölteni a két .csv fájlt a ../data/alá

In [272]:
# .csv file beolvasása
X = pd.read_csv('../data/SpotifyHits.csv')
# Popularity oszlop átmásolva ebbe a fájlba, hogy a modell tanításnál meglegyen a két paraméter
y = pd.read_csv('../data/Y_Popularity.csv')

In [273]:
# Sorok, oszlopok száma X - (Elvárt: 1994 sor, 14 oszlop)
X.shape

(1994, 14)

In [274]:
# Sorok, oszlopok száma Y - (Elvárt: 1994 sor, 2 oszlop)
y.shape

(1994, 2)

In [275]:
# Van-e null érték a DB-ben
X.isnull().sum()

Index           0
Title           0
Artist          0
Genre           0
Year            0
BPM             0
Energy          0
Danceability    0
Loudness        0
Liveness        0
Valence         0
Length          0
Acousticness    0
Speechiness     0
dtype: int64

In [276]:
# Adattípusok ellenőrzése
X.dtypes

Index            int64
Title           object
Artist          object
Genre           object
Year             int64
BPM              int64
Energy           int64
Danceability     int64
Loudness         int64
Liveness         int64
Valence          int64
Length           int64
Acousticness     int64
Speechiness      int64
dtype: object

In [277]:
# Beolvasás sikerességének ellenőrzése
X.head()

,Index,Title,Artist,Genre,Year,BPM,Energy,Danceability,Loudness,Liveness,Valence,Length,Acousticness,Speechiness
0,1,Sunrise,Norah Jones,adult standards,2004,157,30,53,-14,11,68,201,94,3
1,2,Black Night,Deep Purple,album rock,2000,135,79,50,-11,17,81,207,17,7
2,3,Clint Eastwood,Gorillaz,alternative hip hop,2001,168,69,66,-9,7,52,341,2,17
3,4,The Pretender,Foo Fighters,alternative metal,2007,173,96,43,-4,3,37,269,0,4
4,5,Waitin' On A Sunny Day,Bruce Springsteen,classic rock,2002,106,82,58,-5,10,87,256,1,3


In [278]:
# Felesleges oszlopok törlése (főleg szöveges mezők törlése, illetve az Index oszlop felesleges)
X = X.drop(columns=['Index', 'Genre', 'Artist', 'Title'])
y = y.drop(columns=['Index'])
y = y.values.ravel()

In [279]:
# Leíró statisztika létrehozása (törlés utáni állapot)
X.describe()

,Year,BPM,Energy,Danceability,Loudness,Liveness,Valence,Length,Acousticness,Speechiness
count,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000
mean,1992.992979,120.215647,59.679539,53.238215,-9.008526,19.012036,49.408726,262.443330,28.858074,4.994985
std,16.116048,28.028096,22.154322,15.351507,3.647876,16.727378,24.858212,93.604387,29.011986,4.401566
min,1956.000000,37.000000,3.000000,10.000000,-27.000000,2.000000,3.000000,93.000000,0.000000,2.000000
25%,1979.000000,99.000000,42.000000,43.000000,-11.000000,9.000000,29.000000,212.000000,3.000000,3.000000
50%,1993.000000,119.000000,61.000000,53.000000,-8.000000,12.000000,47.000000,245.000000,18.000000,4.000000
75%,2007.000000,136.000000,78.000000,64.000000,-6.000000,23.000000,69.750000,289.000000,50.000000,5.000000
max,2019.000000,206.000000,100.000000,96.000000,-2.000000,99.000000,99.000000,1412.000000,99.000000,55.000000


In [280]:
# Listává konvertálás, obj használata helyett
col = X.columns.tolist()

In [283]:
# Mátrix látrehozása az összes lent megadott oszlop 9sszes rekordjához (One hot Encode features)
onehot_cat = ['Speechiness', 'Energy', 'Danceability', 'Year']
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [285]:
con_cat = [i for i in col if i not in onehot_cat]

In [288]:
cat_pipe = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore')),
    ('imputer', SimpleImputer(strategy='most_frequent', add_indicator=True))
])

In [289]:
con_pipe = Pipeline([
    ('SS', StandardScaler()),
    ('KNN', KNNImputer(n_neighbors=5))
])

In [290]:
preprocessor = ColumnTransformer([
    ('categorical', cat_pipe, onehot_cat),
    ('continuous', con_pipe, con_cat)
])

In [291]:
model_final = BaggingRegressor(n_estimators=50, max_features=1.0, n_jobs=-1)
model_final_pipe = Pipeline([('preprocessor', preprocessor),
                             ('bag', TransformedTargetRegressor(regressor=model_final,
                                                                transformer=QuantileTransformer(n_quantiles=300,
                                                                                                output_distribution='normal')))])

In [292]:
model_final_pipe.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='most_frequent'))]),
                                                  ['Speechiness', 'Energy',
                                                   'Danceability', 'Year']),
                                                 ('continuous',
                                                  Pipeline(steps=[('SS',
                                                                   StandardScaler()),
                                                                  ('KNN',
                                                                   KNNImputer())]),
                                                  ['BPM', 'Loudness',
                                                   'Liveness', 'Valence',
                                                   'Length',
                                                   'Acousticness'])])),
                ('bag',
                 TransformedTargetRegressor(regressor=BaggingRegressor(n_estimators=50,
                                                                       n_jobs=-1),
                                            transformer=QuantileTransformer(n_quantiles=300,
                                                                            output_distribution='normal')))])

In [293]:
pickle.dump(model_final_pipe, open('../data/model_final_trained.pkl', 'wb'))

In [294]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from   sklearn.compose            import *
from   sklearn.experimental       import enable_iterative_imputer
from   sklearn.impute             import *
from   sklearn.metrics            import balanced_accuracy_score # Evaluation metric 2.0 
from   sklearn.pipeline           import Pipeline
from   sklearn.preprocessing      import *
from   sklearn.linear_model    import *
from   sklearn.metrics         import mean_absolute_error
from sklearn.base import BaseEstimator
from   sklearn.tree import *
from   sklearn.ensemble import *
from sklearn.model_selection  import RandomizedSearchCV
from sklearn.model_selection  import GridSearchCV
from sklearn import set_config
from sklearn.model_selection import cross_val_score

In [295]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [296]:
class DummyEstimator(BaseEstimator):
    "Pass through class, methods are present but do nothing."
    def fit(self): pass
    def score(self): pass

In [297]:
n_est = [100, 150] # num of trees (base_estimators)
max_features = [0.8, 1.0] # features in base_estimators
search_space = [{'bag': [BaggingRegressor(n_jobs=-1)], # Actual Estimator
                 'bag__n_estimators': n_est,
                 'bag__max_features': max_features}]

bag_final_pipe = Pipeline([('preprocessor', preprocessor),
                ('bag', TransformedTargetRegressor(regressor=DummyEstimator(),
                                                         transformer=QuantileTransformer(n_quantiles=300,
                                                         output_distribution='normal')))])                

bag_randcv = RandomizedSearchCV(estimator=bag_final_pipe, 
                                 param_distributions=search_space,
                                 cv=5,
                                 scoring=['neg_root_mean_squared_error', 'neg_mean_squared_error'],
                                 n_jobs=-1,
                                 verbose=1,
                                 random_state=42,
                                 refit='neg_root_mean_squared_error')

In [298]:
bag_randcv.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


Fitting 5 folds for each of 4 candidates, totalling 20 fits


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('categorical',
                                                                               Pipeline(steps=[('ohe',
                                                                                                OneHotEncoder(handle_unknown='ignore')),
                                                                                               ('imputer',
                                                                                                SimpleImputer(add_indicator=True,
                                                                                                              strategy='most_frequent'))]),
                                                                               ['Speechiness',
                                                                                'Energy',
                                                                                'Danceability',
                                                                                'Year']),
                                                                              ('continuous',
                                                                               Pipeline(steps=[('SS',
                                                                                                StandardScaler())...
                                                                         transformer=QuantileTransformer(n_quantiles=300,
                                                                                                         output_distribution='normal')))]),
                   n_jobs=-1,
                   param_distributions=[{'bag': [BaggingRegressor(max_features=0.8,
                                                                  n_estimators=100,
                                                                  n_jobs=-1)],
                                         'bag__max_features': [0.8, 1.0],
                                         'bag__n_estimators': [100, 150]}],
                   random_state=42, refit='neg_root_mean_squared_error',
                   scoring=['neg_root_mean_squared_error',
                            'neg_mean_squared_error'],
                   verbose=1)

In [299]:
bag_randcv.best_score_

-13.856491891651023

In [300]:
print(f'Bagging Regressor: {round(bag_randcv.best_score_, 3)}')

Bagging Regressor: -13.856


In [301]:
bag_randcv.best_params_

{'bag__n_estimators': 100,
 'bag__max_features': 0.8,
 'bag': BaggingRegressor(max_features=0.8, n_estimators=100, n_jobs=-1)}

In [302]:
model_final = BaggingRegressor(n_estimators=100, max_features = 0.8, n_jobs=-1)
model_final_pipe = Pipeline([('preprocessor', preprocessor),
                ('bag', TransformedTargetRegressor(regressor=model_final,
                                                         transformer=QuantileTransformer(n_quantiles=300,
                                                         output_distribution='normal')))])
model_final_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='most_frequent'))]),
                                                  ['Speechiness', 'Energy',
                                                   'Danceability', 'Year']),
                                                 ('continuous',
                                                  Pipeline(steps=[('SS',
                                                                   StandardScaler()),
                                                                  ('KNN',
                                                                   KNNImputer())]),
                                                  ['BPM', 'Loudness',
                                                   'Liveness', 'Valence',
                                                   'Length',
                                                   'Acousticness'])])),
                ('bag',
                 TransformedTargetRegressor(regressor=BaggingRegressor(max_features=0.8,
                                                                       n_estimators=100,
                                                                       n_jobs=-1),
                                            transformer=QuantileTransformer(n_quantiles=300,
                                                                            output_distribution='normal')))])

In [305]:
# Itt az abszolút hiba középértékét (MAE) használjuk, mert ezt szeretnénk a dal népszerűségének abszolút hibájával korrelálni.
y_pred = model_final_pipe.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.6f}")

MAE: 10.752468


In [304]:
# Jelenleg nem máködik...
# Grafikus report generálása html fájlba
# Futtatás előtt:
# ! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
#profile = ProfileReport(df, title="Spotify Song Popularity", html={'style':{'full_width':True}})
# Ezt használtam a profiling feature használásához: https://python.plainenglish.io/how-to-use-pandas-profiling-on-google-colab-e34f34ff1c9f

In [ ]:
#profile

In [ ]:
# Report elmentése fájlba
#profile.to_file("../Spotify_Song_Popularity.html")